# Lemke Howson: An algorithm to find Nash Equilibrium

This notebook introduces Lemke Howson algorithm for finding Nash equilibrium of a two-player strategic form game. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Two-Player Games in Strategic Form

We are going to find Nash qquilibria (pure or mixed action) of a Two-Player Games $g = (I, (A_i)_{i \in I}, (u_i)_{i \in I})$, where 

* $I = \{0, 1\}$ is the set of players,
* $M = \{1, ..., m\}, N = \{m+1, ..., m+n\}$ are the action spaces,
* $\Delta^L = \{x \in \mathbb{R}^L_+ \mid \sum_{l \in L} x_l = 1\}, L=M, N$ are the mixed action spaces.
* $A \in \mathbb{R}^{M \times N}, B \in \mathbb{R}^{M \times N}$ are the payoff matrixs for player 0 and 1, respectively, and similarly the expected payoffs are $x^{\prime}Ay$ and $x^{\prime}By$.

### Nash Equilibrium

For $x \in \Delta^M$ and $y \in \Delta^N$, we define

$$
\bar{x} = \underset{j \in N}{\arg \max}(B^{\prime}x)_j, \quad
x^\circ = \{i \in M \mid x_i = 0\} \\
\bar{y} = \underset{i \in M}{\arg \max}(A^{\prime}y)_i, \quad
y^\circ = \{j \in N \mid y_j = 0\} \\
\text{supp}(x) = \{i \mid x_i > 0\}, \quad 
\text{supp}(y) = \{j \mid y_j > 0\}
$$

According to von Stengel, B. (2007), we can judge whether $(x, y) \in \Delta^M \times \Delta^N$ is a Nash equilibrium by checking:

* $\text{supp}(x) \subset \bar{y}, \text{supp}(y) \subset \bar{x}$, or
* $\bar{y} \cup x^\circ = M, \bar{x} \cup y^\circ = N$, equivalently $(\bar{x} \cup x^\circ) \cup (\bar{y} \cup y^\circ) = M \cup N$.

These two conditions are equivalent, so checking one is enough to find a Nash equilibrium.

Depending on these two conditions, we may use some algorithms to compute Nash equilibrium of any well-defined two-player strategic form game. However, we have to deal with some difficulties when facing degenerate games, as we will see later.

We first consider the case with nondegenerate game, which is simpler.

# Nondegenerate game

** Definition: ** A two-player game is nondegenerate if for any $x \in \Delta^M$ and any $y \in \Delta^N$, we have

$$
\left| \bar{x} \right| \leq \left| \text(supp)(x) \right|, \quad
\left| \bar{y} \right| \leq \left| \text(supp)(y) \right|,
$$

or equivalently,

$$
\left| x^\circ \right| + \left| \bar{x} \right| \leq m, \quad
\left| y^\circ \right| + \left| \bar{y} \right| \leq n.
$$

Combined with the condition a Nash equilibrium should satisfy described above, we know that if $(x, y)$ is a Nash equilibrium of a nondegenerate game, then

$$
\left| \text{supp}(x) \right| = \left| \text{supp}(y) \right|
$$

This reduces the number of possible support pairs for a Nash equilibrium a lot.

## Support Enumeration

Before we introduce Lemke Howson algorithm, we introduce a method to find all Nash equilibria by iterating all mixed actions with equal-sized support pairs, and checking whether they can be a Nash equilibrium. This is called Support Enumeration.

For each $k=1, ..., \text{min}\{m, n\}$ and each pair $(I, J)$, $I \subset M$ and $J \subset N$, such that $\left| I \right| = \left| J \right| = k$, the mixed action $(x, y)$ is a Nash equilibrium if it solves the systems of linear equations

$$
\sum_{j \in J} a_{ij} y_j = u \text{ for } i \in I, \quad
\sum_{j \in J} y_j = 1, \\
\sum_{i \in I} b_{ij} x_i = v \text{ for } j \in J, \quad
\sum_{i \in I} x_i = 1
$$

And also satisfies that
* $x_i > 0$ for all $i \in I$ and $y_j >0$ for all $j \in J$,
* $u \geq \sum_{j \in J} a_{ij} y_j$ for all $i \not \in I$ and $v \geq \sum_{i \in I} b_{ij} x_i$ for all $j \not \in J$.

The systems of equations can be written in matrix form as 

$$
C \begin{pmatrix} y_J \\ u \end{pmatrix} = e
$$
and
$$
D \begin{pmatrix} x_I \\ v \end{pmatrix} = e
$$
with
$$
C =
 \begin{pmatrix}
 A_{IJ} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
D =
 \begin{pmatrix}
 B'_{JI} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
e = \begin{pmatrix}\mathbf{0} \\ 1\end{pmatrix},
$$
where
$A_{IJ}$ is the submatrix of $A$ given by rows $I$ and columns $J$,
$B'_{JI}$ is the submatrix of $B'$ given by rows $J$ and columns $I$, and
$\mathbf{0}$ and $\mathbf{1}$ are the $k$-dimensional vectors of zeros and ones, respectively.

Using the algorithm described above, given a well-defined nondegenerate two-player game and a support pair $I, J$, we know whether there is a Nash equilirium corresponding to this support pair, and can calculate the NE $(x, y)$ if there is one.

To make it convenient, we denote the actions by indices, which starts with 0 in Python.

### Code

We first write a function which solves the system of linear equations for one player, which is the half of the whole system to find a Nash equilibiurm. In other word, it checks whether there exists a mixed action of opponent with support given, against which the actions in the player's own support are all best responses.

* The arguments are
  * numpy array `payoff_matrix`;
  * list (or numpy array) `own_supp` for the support of the player in consideration;
  * list (or numpy array) `opp_supp` for the support of the opponent player;
  * numpy array `out` that stores the candidate mixed action.
* If there is a mixed action of the opponent with support `opp_supp`
  against which the actions in `own_supp` are best responses,
  then return `True`; otherwise return `False`.
  In the former case, the mixed action is stored in `out`.
* Array `out` must be of length equal to the number of the opponent's actions.

In [2]:
def indiff_mixed_action(payoff_matrix, own_supp, opp_supp, out):
    # (number of own actions, number of opponent's actions)
    nums_actions = payoff_matrix.shape
    
    # Support size
    k = len(own_supp)
    
    # Matrix in the left hand side of the linear equation
    a = np.empty((k+1, k+1))
    a[:-1, :-1] = payoff_matrix[own_supp, :][:, opp_supp]
    a[-1, :-1] = 1
    a[:-1, -1] = -1
    a[-1, -1] = 0
    
    # Vector in the right hand side of the linear equation
    b = np.zeros(k+1)
    b[-1] = 1
    
    try:
        sol = np.linalg.solve(a, b)
    except np.linalg.LinAlgError:
        return False
    
    # Return False immediately if any of the "probabilities" is not positive
    if (sol[:-1] <= 0).any():
        return False
    
    own_supp_c = np.setdiff1d(np.arange(nums_actions[0]), own_supp)
    # Return False immediately if the solution mixed action is not optimal
    if (sol[-1] < payoff_matrix[own_supp_c, :][:, opp_supp] @ sol[:-1]).any():
        return False
    
    out.fill(0)
    out[opp_supp] = sol[:-1]
    return True

Given support $I$ and $J$, if for each player, the `indiff_mixed_action` returns `True`, then there is a Nash equilirium with support $I$ and $J$, and the mixed actions are just as calculated.

To find all Nash equilibria of a strategic form game, we just iterate all possible combinations of $I$ and $J$, and then apply `indiff_mixed_action` to each players with each support pair. If `indiff_mixed_action` returns `True` for both players, we store mixed actions in list `NEs`.

In [3]:
import itertools

def support_enumeration(A, B_T):
    """
    Given a nondegenerate m x n bimatrix game (A, B_T), return a list of
    all Nash equilibria computed by the support enumeration algorithm.
    
    Parameters
    ----------
    A : ndarray(float, ndim=2)
        Payoff matrix for player 0, of shape (m, n).
    
    B_T : ndarray(float, ndim=2)
        Payoff matrix for player 1, of shape (n, m).
        
    Returns
    -------
    NEs : list(tuple(ndarray(float, ndim=1)))
        List containing tuples of Nash equilibrium mixed actions.
    
    """
    m, n = A.shape
    NEs = []
    for k in range(1, min(m, n)+1):
        for I in itertools.combinations(range(m), k):
            for J in itertools.combinations(range(n), k):
                y = np.empty(n)
                if indiff_mixed_action(A, list(I), list(J), y):
                    x = np.empty(m)
                    if indiff_mixed_action(B_T, list(J), list(I), x):
                        NEs.append((x, y))
    return NEs

### Example

Consider the example from von Stengel (2007):
$$
A =
\begin{bmatrix}
3 & 3 \\
2 & 5 \\
0 & 6
\end{bmatrix},
\quad
B =
\begin{bmatrix}
3 & 2 \\
2 & 6 \\
3 & 1
\end{bmatrix}.
$$

The action spaces of players 0 and 1 are
$$
M = \{0, 1, 2\}, \quad
N = \{0, 1\}.
$$

Use `indiff_mixed_action` to find a Nash equilibrium, when $I = \{0, 1\}$ and $J = \{0, 1\}$.

In [4]:
# Define a two-player strategic form game
A = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
B = np.array([[3, 2],
              [2, 6],
              [3, 1]])
m, n = A.shape  # Numbers of actions of players 0 and 1, respectively
M = np.arange(m)
N = np.arange(n)

# Set the equal-sized support pari I, J
I = [0, 1]
J = [0, 1]

out = np.empty(n)
indiff_mixed_action(A, I, J, out)

True

In [5]:
out

array([ 0.66666667,  0.33333333])

Use `support_enumeration`, we can find all Nash equilibria.

In [6]:
support_enumeration(A, B.T)

[(array([ 1.,  0.,  0.]), array([ 1.,  0.])),
 (array([ 0.8,  0.2,  0. ]), array([ 0.66666667,  0.33333333])),
 (array([ 0.        ,  0.33333333,  0.66666667]),
  array([ 0.33333333,  0.66666667]))]

In this game, there are 3 NEs.

Support Enumeration has one drawback, that the number of equal-sized support pairs increases quickly as the game gets larger:

If $m = n$, the number of equal-sized support pairs is 

$$
\sum^n_{k=1} \binom{n} {k} ^2 = \binom {2n} {n} - 1 \approx \frac{4^n}{\sqrt{\pi n}}
$$

We can see that the number of equal-sized support pairs increase exponentially. That's why Lemke-Howson algorithm is more desirable, which is less computational intensive.

## Lemke Howson

### Polyhedrons and Labels

Given a bimatrix game with payoff matrixs being $A$ and $B$ for player 0, 1 respectively, the best response can be represented as a polyhedron:

$$
\bar{P} = \{(x, v) \in \mathbb{R}^M \times \mathbb{R} \mid x\geq \mathbf{0}, B^{\prime}x \leq v \mathbf{1}, \mathbf{1}^{\prime}x = 1\} \\
\bar{Q} = \{(y, u) \in \mathbb{R}^N \times \mathbb{R} \mid y\geq \mathbf{0}, A^{\prime}y \leq u \mathbf{1}, \mathbf{1}^{\prime}y = 1\}
$$

where $x \in \Delta^M$, $y \in \Delta^N$. $v$ is the upper bound of expected payoffs for player 1 choosing pure actions when player 0's mixed action is $x$. Similarly, $u$ is the upper bound of expected payoffs for player 0.  

Define the *label* of a point in best response polyhedron as:  

* $(x, v) \in \bar{P}$ has *label* $k \in M \cup N$ if 
 * for $k = j \in N$, $(B^{\prime}x)_{j} = v$, so that $j \in \bar{x}$, or,
 * for $k = i \in M$, $x_i = 0$, so that $i \in x^{\circ}$.
 
* $(y, u) \in \bar{Q}$ has *label* $k \in M \cup N$ if 
 * for $k = i \in M$, $(A^{\prime}y)_{i} = u$, so that $i \in \bar{y}$, or,
 * for $k = j \in N$, $y_j = 0$, so that $j \in y^{\circ}$.
 
Without loss of generality, we assume that $A$ and $B^{\prime}$ are nonnegative and have no zero column. Dividing by $v$ and $u$, we can trun the best response polyhedron into a best response polytope:

$$
P = \{x \in \mathbb{R}^M \mid x \geq \mathbf{0}, B^{\prime}x \leq 1 \} \\
Q = \{y \in \mathbb{R}^N \mid y\geq \mathbf{0}, A^{\prime}y \leq 1\}
$$

Using the same example as in the Support Enumeration, where 

$$
M = \{1, 2, 3\}, \quad
N = \{4, 5\}. \\
A =
\begin{bmatrix}
3 & 3 \\
2 & 5 \\
0 & 6
\end{bmatrix},
\quad
B =
\begin{bmatrix}
3 & 2 \\
2 & 6 \\
3 & 1
\end{bmatrix}.
$$

We can draw the best response polytope as follows.

(Note that here the actions are indexed as $\{1, 2, 3\}$ and $ \{4, 5\}$ to prevent ambiguity. When we implement the Lemke Howson algorithm using Python, it will be indexed by $\{0, 1, 2\}$ and $ \{0, 1\}$ as before.)

![Lemke_Howson_Fig1](https://raw.githubusercontent.com/shizejin/Lemke_Howson_notebook/master/Lemke_Howson_Fig1.png)

$((x, v), (y, u)) \in \bar{P} \times \bar{Q}$ is *completely labeled* if every $k \in M \cup N$ appears as a label of either $(x, v)$ or $(y, u)$.

In the context of best response polyhedron and labels, we can define Nash equilibrium as:  
* $(x, y) \in \Delta^{M} \times \Delta^{N}$ is a Nash equilibrium if and only if $((x, v), (y, u))$ with $u = \text{max}_i (Ay)_i$ and $v = \text{max}_j (B^{\prime}x)_j$ is completely labeled.

The transformed best response polytope is a special case where $u = v = 1$, and we have

* $(x, y) \in P \times Q$, $(x, y) \neq (\mathbf{0}, \mathbf{0})$, is an ("un-normalized") Nash equilibirum if and only if $(x, y)$ is completely labeled.

If the game is nondegenerate, it is easy to know that in $P(Q)$, only vertices can have $m(n)$ labels.

Thus, the Nash equilibrium is confined in vertice pairs of two best response polyhedra. We can find a Nash equilibrium by enumerating vertice pairs to find completely labeled one. However, Lemke Howson algorithm provides a more efficient way that avoids enumerating.

### Lemke Howson algorithm for finding completely labeled vertice pairs

By all defintion, $(\mathbf{0}, \mathbf{0}) \in P \times Q$ is completely labeled. However, it is not a Nash equilibrium. We call it "artificial equilibrium". The Lemke Howson algorithm starts from $(\mathbf{0}, \mathbf{0})$, and moves the vertice by replacing the labels in $P$ and $Q$ in turn until it is completely labeled.

In detail, in the same example as before:

* $(\mathbf{0}, \mathbf{0})$ has labels {1, 2, 3}, {4, 5}
* $(\mathbf{0}, \mathbf{0}) \rightarrow (c, \mathbf{0})$ with label {1, 3, 5}, {4, 5}  
 Drop Label 2 in $P$ (Label to drop is arbitrarily chosen in the first step). Label 5 is picked up.
* $(c, \mathbf{0}) \rightarrow (c, p)$ with label {1, 3, 5}, {3, 4}  
 Drop Label 5 in $Q$ as it is label of both $x$ and $y$. Label 3 is picked up.
* $(c, p) \rightarrow (d, p)$ with label {1, 4, 5}, {3, 4}  
 Drop Label 3 in $P$ as it is label of both $x$ and $y$. Label 4 is picked up.
* ... ...
* Continue the procedure, until $(d, q)$ is achieved, which is completely labeled, and thus is a un-normalized Nash equilibrium.

### Complementary Pivoting

To implementing the dropping and picking up of labels, we introduce slack variables $s_4, s_5, r_1, r_2, r_3$:

$$
\begin{matrix}
3 x_1 & + 2 x_2 & + 3 x_3 & + s_4 & & = 1\\
2 x_1 & + 6 x_2 & + x_3 & & + s_5 & = 1 \\
\\
r_1 & & & + 3 y_4 & + 3 y_5 & = 1\\
 & r_2 & & + 2 y_4 & + 5 y_5 & = 1 \\
 & & r_3 & & + 6 y_5 & = 1 
\end{matrix}
$$

$x \geq \mathbf{0}, s \geq \mathbf{0}, r \geq \mathbf{0}, y \geq \mathbf{0}.$

A solution $(x, s, r, y)$ is completely labeled if and only if  
$$
x^\prime r = 0, \quad y^\prime s = 0
$$

Variables in $(x, s, r, y)$ are called *basic variables* if they are positive, and *nonbasic variables* if they are equal to $0$.

Therefore, the movement of vertice point is equivalent to *Pivoting*, which is a change of the basis where a nonbasic variable enters and a basic variable leaves the set of basic variables while satisfying the nonnegativity constraint.

During the process, what we need to record are the indexes of basic variables and coefficients of linear equations systems. We denote the coefficient matrix by *tableau*, which initially are: 

$$
tableau_1 = 
\begin{bmatrix}
3 & 2 & 3 & 1 & 0 & 1 \\
2 & 6 & 1 & 0 & 1 & 1 \\
\end{bmatrix}
$$

and 

$$
tableau_2 =
\begin{bmatrix}
1 & 0 & 0 & 3 & 3 & 1 \\
0 & 1 & 0 & 2 & 5 & 1 \\
0 & 0 & 1 & 0 & 6 & 1 
\end{bmatrix}
$$

respectively.

In detail, the algorithm is as follows:

1. Input two tableaus and lists of basic variables. As we start from $(\mathbf{0}, \mathbf{0})$, the initial basic variables are $\{s_4, s_5\}$ and $\{r_1, r_2, r_3\}$.
2. Start with pivoting in $P$. Initial pivot index can be arbitrarily chosen. Because nonnegativity constraint, the basic variable to be replaced is decided by minimum ratio test. Then update the tableau and basis of $P$ by dropping and adding basic variables.
3. Do pivoting in $Q$. Check whether the newly dropped basic variable has the same index with the first added basic variable by initial pivoting. If it is, then the solution is completely labeled, and a Nash equilibrium is found.
4. Repeat 3 and 4, until a Nash equilibirum is found. (Note that we will not be in infinite loop, as the existence of Nash equilibrium is guaranteed.)

### Code

We simply implement Lemke Howson algorithm using Python, with the same nondegenerate game. (Note that the indexes are changed to be consistent with the 0-based indexing in Python.)

First step, we create tableaus and index lists of basic variables. Note that the index list of basic variables is corresponding to the rows of tableau in order.

In [7]:
# Player 0
#create tableau of Player0
tableau0 = np.empty((n, m+n+1))
tableau0[:, :m] = B.T
tableau0[:, m:m+n] = np.identity(n)
tableau0[:, -1] = 1

# the indexes of basic variables of Player0 at (0, 0)
basic_vars0 = np.arange(m, m+n)

# Player 1
#create tableau of Player1
tableau1 = np.empty((m, n+m+1))
tableau1[:, :m] = np.identity(m)
tableau1[:, m:m+n] = A
tableau1[:, -1] = 1

# the indexes of basic variables of Player1 at (0, 0)
basic_vars1 = np.arange(0, m)

In [8]:
tableau0, tableau1

(array([[ 3.,  2.,  3.,  1.,  0.,  1.],
        [ 2.,  6.,  1.,  0.,  1.,  1.]]),
 array([[ 1.,  0.,  0.,  3.,  3.,  1.],
        [ 0.,  1.,  0.,  2.,  5.,  1.],
        [ 0.,  0.,  1.,  0.,  6.,  1.]]))

In [9]:
basic_vars0, basic_vars1

(array([3, 4]), array([0, 1, 2]))

Define `min_ratio_test` function for deciding leaving basic variable, when basic variable with pivot index is entering.

In [10]:
def min_ratio_test(tableau, pivot):
    
    ind_nonpositive = tableau[:, pivot] <= 0
    # we suppress the "divide by zero" warning message
    with np.errstate(divide='ignore', invalid='ignore'):
        ratios = tableau[:, -1] / tableau[:, pivot]
    # leave out the pivots that have negative ratio
    ratios[ind_nonpositive] = np.inf
    # find the pivot with minimum ratio, under nonnegativity condition
    row_min = ratios.argmin()
    
    return row_min

`Pivoting` function is for updating the tableau after we decide entering and leaving basic variables. 

In [11]:
def pivoting(tableau, pivot, pivot_row):
    """
    Perform a pivoting step.
    
    Parameters
    ----------
    tableau : ndarray(float, ndim=2)
            The tableau to be updated
    
    pivot : int64
            The index of entering basic variable
            
    pivot_row: int64
            The row indice of tableau chosen by minimum ratio test
            the corresponding basic variable is going to be dropped
    
    Returns
    -------
    tableau : ndarray(float, ndim=2)
            The updated tableau
            
    """
    # Row indices except pivot_row
    ind = np.ones(tableau.shape[0], dtype=bool)
    ind[pivot_row] = False
    
    # Store the values in the pivot column, except for row_min
    # Made 2-dim by np.newaxis
    multipliers = tableau[ind, pivot, np.newaxis]
    
    # Update the tableau
    tableau[pivot_row, :] /= tableau[pivot_row, pivot]
    tableau[ind, :] -= tableau[pivot_row, :] * multipliers
    
    return tableau

In [12]:
def lemke_howson_tbl(tableau0, tableau1, basic_vars0, basic_vars1, init_pivot):
    """
    Wrap the procedure of Lemke Howson algorithm.
    
    Parameters
    ----------
    tableau0 : ndarray(float, ndim=2)
            The tableau of Player 0
    tableau1 : ndarray(float, ndim=2)
            The tableau of Player 1
    basic_vars0 : int64
            The index list of basic variables of tableau0
    basic_vars1 : int64
            The index list of basic variables of tableau1
    init_pivot : int64
            The initial pivot index, can be arbitrarily chosen
            
    Returns
    -------
    out : list(float, ndim=1)
        List of concatenated mixed action Nash equilibrium(un-normalized)
        of two Players
            
    """
    m, n = tableau1.shape[0], tableau0.shape[0]
    tableaux = (tableau0, tableau1)
    basic_vars = (basic_vars0, basic_vars1)
    init_player = int((basic_vars[0]==init_pivot).any())
    players = [init_player, 1 - init_player]

    pivot = init_pivot

    while True:
        for i in players:
            # Determine the leaving variable
            row_min = min_ratio_test(tableaux[i], pivot)
            
            # Pivoting step: modify tableau in place
            pivoting(tableaux[i], pivot, row_min)
            
            # Update the basic variables and the pivot
            basic_vars[i][row_min], pivot = pivot, basic_vars[i][row_min]
            
            if pivot == init_pivot:
                break
        else:
            continue
        break

    out_dtype = np.result_type(*tableaux)
    out = np.zeros(m+n, dtype=out_dtype)
    for i, (start, num) in enumerate(zip((0, m), (m, n))):
        ind = basic_vars[i] < start + num if i == 0 else start <= basic_vars[i]
        out[basic_vars[i][ind]] = tableaux[i][ind, -1]
        
    return out

In [13]:
init_pivot = 1
lemke_howson_tbl(tableau0, tableau1, basic_vars0, basic_vars1, init_pivot)

array([ 0.        ,  0.125     ,  0.25      ,  0.08333333,  0.16666667])

By the un-normalized NE returned by `lemke_howson_tbl`, we can calculate the normalized NE:

$$
x = (0, \frac{1}{3}, \frac{2}{3}) \quad y = (\frac{1}{3}, \frac{2}{3}) \\
u = \frac{8}{3}, \quad v = 4
$$

We can use the `lemke_howson` routine from `quantecon.py`, which wraps the procedure of creating tableaus and index lists of basic variables, pivoting, and normalizing the found Nash equilibrium. 

In [14]:
from quantecon.game_theory import lemke_howson, NormalFormGame, Player

# create normal form game
g = NormalFormGame((Player(A), Player(B.T)))
# use lemke_howson with initial pivot being 1
lemke_howson(g, init_pivot = 1)

(array([ 0.        ,  0.33333333,  0.66666667]),
 array([ 0.33333333,  0.66666667]))

# Degenerate Games

In a degenerate game, the minimum ratio test may have more than one minimizers. In this case, arbitrary tie breaking may lead to cycling, so that the algorithm falls into an infinite loop.

For example, we change the payoff matrixes for Player 0 and 1 to $C$ and $D$:

$$
C =
\begin{bmatrix}
3 & 3 \\
2 & 5 \\
0 & 6
\end{bmatrix},
\quad
D =
\begin{bmatrix}
3 & 3 \\
2 & 6 \\
3 & 1
\end{bmatrix}.
$$

As `numpy.argmin()` returns the first indice when there is a tie in 1-dimensional array, if we still use `lemke_howson_tbl` which utilize `min_ratio_test`, it may fail to find a Nash equilirbium properly.

We use lexico-minimum ratio test to replace minimum ratio test in a degenerate game for determining the leaving basic variable.

Note that the original system can be written as 

$$
B^\prime x + I s = \mathbf{1}
$$

where $x$ is the mixed action, and $s$ is the slack variable vector.

The lexico-minimum ratio test introduce $(\epsilon, \epsilon^1, \cdots, \epsilon^n)^\prime$ to the right hand:

$$
B^\prime x + I s = \mathbf{1} + (\epsilon, \epsilon^1, \cdots, \epsilon^n)^\prime
$$

After any number of pivoting steps, the system can be representd by pre-multiplying a inverse of a basic matrix $C$:

$$
C B^\prime x + C I s = C \mathbf{1} + C (\epsilon, \epsilon^1, \cdots, \epsilon^n)^\prime
$$

Write $c_{i0} + c_{i1} ε^1 + ··· + c_{in} ε^n$ for the ith entry of the vector in the right hand, and let $d_i$ be the ith row of the pivoting column.

The lexico-minimum ratio test break a tie by comparing the $c_{ik}$ and $c_{jk}$ in order:

* Choose the minimizers of $c_{i0} / d_{i}$.
* If more than one, among them choose the minimizers of $c_{i1} / d_{i}$.
* repeat this until there is only one minimizer

Note that when implementing this in code, the matrix $C$ is the same with the coefficient matrix of slack variables, and thus we do not need to extend tableau to record more information.

## code

In [15]:
def min_ratio_test_no_tie_breaking(tableau, pivot, test_col, argmins, num_argmins):
    """
    Given the column of coefficient matrix of slack variable, C,
    calculate the number of minimizers.
    
    Parameters
    ----------
    tableau : ndarray(float, ndim=2)
            The tableau contains coefficient matrix C
    
    pivot : int64
            The index of entering basic variable
            
    test_col : int64
            The column indice of coefficient matrix which is going
            to be used as numerator of lexico-minimum ratio
            
    argmins : list(int64, ndim=1)
            The list stores the indices of minimizers
            
    num_argmins : int64
            The length of "argmins" list
    
    Returns
    -------
    idx : int64
            The number of minizers
            
    """
    idx = 0
    i = argmins[idx]
    if tableau[i, pivot] > 0:
        min_ratio = tableau[i, test_col] / tableau[i, pivot]
    else:
        min_ratio = np.inf
    for k in range(1, num_argmins):
        i = argmins[k]
        if tableau[i, pivot] <= 0:
            continue
        ratio = tableau[i, test_col] / tableau[i, pivot]
        if ratio > min_ratio:
            continue
        elif ratio < min_ratio:
            min_ratio = ratio
            idx = 0
        elif ratio == min_ratio:
            idx += 1
        argmins[idx] = k
        
    return idx + 1

In [16]:
def lex_min_ratio_test(tableau, pivot, slack_start):
    """
    Implementing lexico-minimum ratio test to find the 
    only leaving basic variable.
    
    Parameters
    ----------
    tableau : ndarray(float, ndim=2)
            The tableau contains coefficient matrix C
    
    pivot : int64
            The index of entering basic variable
            
    slack_start : int64
            The column indice that coefficient matrix of 
            slack variable starts
    
    Returns
    -------
    agrmins[0] : int64
            The row indice of leaving basic variable
            
    """
    num_rows = tableau.shape[0]
    argmins = np.arange(num_rows)
    num_argmins = num_rows
    
    num_argmins = min_ratio_test_no_tie_breaking(tableau, pivot, -1, argmins, num_argmins)
    if num_argmins == 1:
        return argmins[0]
    
    for j in range(slack_start, slack_start+num_rows):
        if j == pivot:
            continue
        num_argmins = min_ratio_test_no_tie_breaking(tableau, pivot, j, argmins, num_argmins)
        if num_argmins == 1:
            break
            
    return argmins[0]

In [17]:
def lemke_howson_tbl_lex_min(tableau0, tableau1, basic_vars0, basic_vars1, init_pivot):
    """
    The same with `lemke_howson_tbl` other than that 
    we use lexico-minimum ratio test to break a tie.
    
    """
    m, n = tableau1.shape[0], tableau0.shape[0]
    tableaux = (tableau0, tableau1)
    basic_vars = (basic_vars0, basic_vars1)
    init_player = int((basic_vars[0]==init_pivot).any())
    players = [init_player, 1 - init_player]

    pivot = init_pivot
    slack_starts = (m, 0)

    while True:
        for i in players:
            # Determine the leaving variable
            row_min = lex_min_ratio_test(tableaux[i], pivot, slack_starts[i])
            
            # Pivoting step: modify tableau in place
            pivoting(tableaux[i], pivot, row_min)
            
            # Update the basic variables and the pivot
            basic_vars[i][row_min], pivot = pivot, basic_vars[i][row_min]
            
            if pivot == init_pivot:
                break
        else:
            continue
        break

    out = np.zeros(m+n)
    for i, (start, num) in enumerate(zip((0, m), (m, n))):
        ind = basic_vars[i] < start + num if i == 0 else start <= basic_vars[i]
        out[basic_vars[i][ind]] = tableaux[i][ind, -1]
        
    return out

In [18]:
# payoff matrixes of a degenerate game
C = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
D = np.array([[3, 3],
              [2, 6],
              [3 ,1]])

# Player 0
#create tableau of Player0
tableau0 = np.empty((n, m+n+1))
tableau0[:, :m] = D.T
tableau0[:, m:m+n] = np.identity(n)
tableau0[:, -1] = 1

# the indexes of basic variables of Player0 at (0, 0)
basic_vars0 = np.arange(m, m+n)

# Player 1
#create tableau of Player1
tableau1 = np.empty((m, n+m+1))
tableau1[:, :m] = np.identity(m)
tableau1[:, m:m+n] = C
tableau1[:, -1] = 1

# the indexes of basic variables of Player1 at (0, 0)
basic_vars1 = np.arange(0, m)

In [19]:
lemke_howson_tbl_lex_min(tableau0, tableau1, basic_vars0, basic_vars1, init_pivot=1)

array([ 0.        ,  0.125     ,  0.25      ,  0.08333333,  0.16666667])

Note that the `lemke_howson` routine in `quantecon.py` uses lexico-minimum ratio test, so there is no need to concern about the nondegenerate game when using it.

In [20]:
g = NormalFormGame((Player(C), Player(D.T)))
lemke_howson(g, init_pivot = 1)

(array([ 0.        ,  0.33333333,  0.66666667]),
 array([ 0.33333333,  0.66666667]))

# Introduce how to use the `lemke_howson` routine in QuantEcon.py, and use Codenotti, De Rossi, and Pagan's simulations as an example.